In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc
from datetime import datetime
from time import sleep
import os
from filter import filter_links, filter_and_extract_gb_from_df



## Tạo hàm chuyển đổi Cookies

In [7]:
def convert_cookie_string_to_dict(cookie_string):
    cookie_dict = {}
    items = cookie_string.split(';')
    for item in items:
        if '=' in item:  # Kiểm tra xem có dấu '=' trong item không
            key, value = item.strip().split('=', 1)
            cookie_dict[key] = value
    return cookie_dict

a = "REC_T_ID=00242cbd-931c-11ee-be6d-ba8ff0e0943f;SPC_F=3X0jeS85aHy10x5rodCSg42OXHfLudq9;_fbp=fb.1.1701745895419.1410008333;_gcl_aw=GCL.1701849803.CjwKCAiA1MCrBhAoEiwAC2d64XlvnagyGDA_3wu44TfZHPzZycZGG3fbRSrosZ2jxQLcZu-muHFWsRoCR9MQAvD_BwE;_gac_UA-61914164-6=1.1701849804.CjwKCAiA1MCrBhAoEiwAC2d64XlvnagyGDA_3wu44TfZHPzZycZGG3fbRSrosZ2jxQLcZu-muHFWsRoCR9MQAvD_BwE;_ga_M32T05RVZT=GS1.1.1701847999.2.1.1701849865.60.0.0;_hjSessionUser_868286=eyJpZCI6IjE2MzcyZjc5LTNkNDYtNTkwZC1iNjI0LTEwMTMwNTJlMTUzNyIsImNyZWF0ZWQiOjE3MDE3NDYxMDU4NTQsImV4aXN0aW5nIjp0cnVlfQ==;SPC_CLIENTID=M1gwamVTODVhSHkxetweujfvqupjahmc;SPC_U=142941210;SPC_R_T_ID=7DIrYSWuqZNQCmrHD0vvMOS7nwq1JJzyUMdnI55xuGflPZqucWoB1rMqzFsFrS3iIYLz0TSSswYfzMXTXX0ELPimrAxzHKKFCq5T+VQrF29SfDfH9y6ckMA+LafFh2ULvbflFkXHRgtU6QVarqxW6NPrbOPh9zajs9ucON0+ROM=;SPC_R_T_IV=NWZTQ2RzS3NvdG41WkxrYg==;SPC_T_ID=7DIrYSWuqZNQCmrHD0vvMOS7nwq1JJzyUMdnI55xuGflPZqucWoB1rMqzFsFrS3iIYLz0TSSswYfzMXTXX0ELPimrAxzHKKFCq5T+VQrF29SfDfH9y6ckMA+LafFh2ULvbflFkXHRgtU6QVarqxW6NPrbOPh9zajs9ucON0+ROM=;SPC_T_IV=NWZTQ2RzS3NvdG41WkxrYg==;SPC_SI=CFPUZQAAAABobmlYWW9XRJYTSgIAAAAANDNJOWlTNnk=;_gid=GA1.2.1848293112.1709302728;SPC_SC_TK=a73f109649c3cfeef88c5b0546dca094;SPC_SC_UD=142941210;SPC_STK=yYZV1WBX6CrluS344WO0VBeOBrG7kA/mXOA8TxayL81H7pxeLMeQYTD4dXc8ZPCRppkw5qkK6zOCnqgvRJJVmrTlcCtzeH00k+pKpiZEsZupMDi6XX+eMeHv2qO8sIMqDTpy9R7O2su8/0uJzu2t9v0SCZ+1a0MmRcdDTzFLCKDEZVTUe8FOZA3VlWkFoGpT;SC_DFP=TaIeyIvQNeXtGVmLjdYrKOsPNApiradQ;_ga_3XVGTY3603=GS1.1.1709352133.3.1.1709352391.60.0.0;_med=refer;SPC_ST=.TjhRWnNIeHQwNmJPbzhtV1c9us67S3wXJBDGIWqIt38u4piZk+MGAlvUoVrnmJ6LQa8tjOtjP4a0VhDSkIU/SgqhiOgG5Rm3j2MlIopJgDNtdof1dPy0sBWdFwaMpNHl5E6p6no2k7LU66tNnF9zQZhLVCuDUWcW5BDqHq+VdLoy6jeIIV4BewStYrQ0H8yPE14aq8A5CbzWTy09gtHyfA==;SPC_EC=.TjhRWnNIeHQwNmJPbzhtV1c9us67S3wXJBDGIWqIt38u4piZk+MGAlvUoVrnmJ6LQa8tjOtjP4a0VhDSkIU/SgqhiOgG5Rm3j2MlIopJgDNtdof1dPy0sBWdFwaMpNHl5E6p6no2k7LU66tNnF9zQZhLVCuDUWcW5BDqHq+VdLoy6jeIIV4BewStYrQ0H8yPE14aq8A5CbzWTy09gtHyfA==;_gcl_au=1.1.673493001.1709522456;SPC_SEC_SI=v1-T2dvZ0pyMk5IQXhRenhobNzAhdsnAqzvamLlnwRYmqoqcpRO7uFFpT3ZDSaLY6Al3LwzV+EF4o1+GWIOCNmEW/OjCoFLXRZ39ukv399o5sE=;__LOCALE__null=VN;csrftoken=osoKLpMpBRnLk1ZJubyMtiiypXMft1gC;_sapid=9509f6ffb058729a5c3884ce9ced2f8f1605b70813d4c7dd341f156c;_QPWSDCXHZQA=6efefc3e-78da-4e0b-e40d-8aeb6a9f2d17;REC7iLP4Q=966f9dcc-dbef-4b80-9572-a74b08014da0;shopee_webUnique_ccd=4ZM9gaGx9nK3n8qbarbxFQ%3D%3D%7Coy1vXutpoq8Bj7hXRG8pHqv8kjAhM7wuxhIptGaUIJDijm1X%2BoPovWHJjs50N6WVJCrbwUHEncs%3D%7C0tJua14Do9l%2FO5v3%7C08%7C3;ds=4e19fe6ee89a2e396d36a6b9076bd3a9;_hjSession_868286=eyJpZCI6IjdmZGJhN2EzLTYyMmQtNGZhZC05ZWZkLWNmMzE1NzNiOTRlOSIsImMiOjE3MDk2MDMxNjQwNTAsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=;_ga_4GPP1ZXG63=GS1.1.1709603164.48.0.1709603164.60.0.0;AMP_TOKEN=%24NOT_FOUND;_ga=GA1.2.1033809469.1701745897;_dc_gtm_UA-61914164-6=1;|Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
# Sử dụng hàm đã định nghĩa để chuyển đổi
cookies = convert_cookie_string_to_dict(a)
cookies

{'REC_T_ID': '00242cbd-931c-11ee-be6d-ba8ff0e0943f',
 'SPC_F': '3X0jeS85aHy10x5rodCSg42OXHfLudq9',
 '_fbp': 'fb.1.1701745895419.1410008333',
 '_gcl_aw': 'GCL.1701849803.CjwKCAiA1MCrBhAoEiwAC2d64XlvnagyGDA_3wu44TfZHPzZycZGG3fbRSrosZ2jxQLcZu-muHFWsRoCR9MQAvD_BwE',
 '_gac_UA-61914164-6': '1.1701849804.CjwKCAiA1MCrBhAoEiwAC2d64XlvnagyGDA_3wu44TfZHPzZycZGG3fbRSrosZ2jxQLcZu-muHFWsRoCR9MQAvD_BwE',
 '_ga_M32T05RVZT': 'GS1.1.1701847999.2.1.1701849865.60.0.0',
 '_hjSessionUser_868286': 'eyJpZCI6IjE2MzcyZjc5LTNkNDYtNTkwZC1iNjI0LTEwMTMwNTJlMTUzNyIsImNyZWF0ZWQiOjE3MDE3NDYxMDU4NTQsImV4aXN0aW5nIjp0cnVlfQ==',
 'SPC_CLIENTID': 'M1gwamVTODVhSHkxetweujfvqupjahmc',
 'SPC_U': '142941210',
 'SPC_R_T_ID': '7DIrYSWuqZNQCmrHD0vvMOS7nwq1JJzyUMdnI55xuGflPZqucWoB1rMqzFsFrS3iIYLz0TSSswYfzMXTXX0ELPimrAxzHKKFCq5T+VQrF29SfDfH9y6ckMA+LafFh2ULvbflFkXHRgtU6QVarqxW6NPrbOPh9zajs9ucON0+ROM=',
 'SPC_R_T_IV': 'NWZTQ2RzS3NvdG41WkxrYg==',
 'SPC_T_ID': '7DIrYSWuqZNQCmrHD0vvMOS7nwq1JJzyUMdnI55xuGflPZqucWoB1rMqzFsFrS3iIYLz0TSSswY

## Links Sản Phẩm

In [3]:
def add_cookies(driver, cookies):
    driver.get("https://shopee.vn")  # Mở Shopee để thêm cookies
    sleep(3)  # Đợi trang tải xong
    for name, value in cookies.items():
        driver.add_cookie({'name': name, 'value': value})
    driver.refresh()  # Tải lại trang sau khi thêm cookies
    print("Cookies added successfully.")

def safe_find_element(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element
    except NoSuchElementException:
        return None

def get_product_links(driver, j):
    product_links = []  # Danh sách để lưu trữ các link sản phẩm
    url = f'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.11036030.11036031?brands=1695294&page={j}&sortBy=pop'
    driver.get(url)
    sleep(4.765)  # Adjust based on your actual wait time needed for the page to load

    try:
        for i in range(1, 61):  # Thay đổi số lượng lặp nếu cần
            try:
                # Cập nhật selector để phản ánh đúng cấu trúc của trang web
                selector = f"#main > div > div:nth-child(3) > div > div > div.container.cZOmU2 > div.uXYBTq > section > ul > li:nth-child({i}) > a"
                element = driver.find_element(By.CSS_SELECTOR, selector)
                product_link = element.get_attribute('href')
                if product_link:
                    product_links.append(product_link)
                    print(product_link)
            except NoSuchElementException:
                print(f"Phần tử thứ {i} không được tìm thấy.")
                continue  # Dừng vòng lặp nếu phần tử không được tìm thấy

        return product_links
    except TimeoutException:
        print("Timed out waiting for page to load")
        return []


In [4]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

# Thêm cookies vào trình duyệt
add_cookies(driver, cookies)

product_links = []
for j in range(0,17):  # Giả sử bạn chỉ muốn lấy dữ liệu từ trang đầu tiên
    product_links.extend(get_product_links(driver, j))

driver.quit()

# Lọc ra các link sản phẩm của iPhone 15
links_smartphone = filter_links(product_links)

# Xuất dữ liệu ra file CSV
products_df = pd.DataFrame(links_smartphone, columns=['URL'])
links = products_df['URL'].to_list()
print(links)

Cookies added successfully.
https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Samsung-Galaxy-A14-5G-(4GB-128GB)-H%C3%A0ng-ch%C3%ADnh-h%C3%A3ng-i.85762618.23981430230?sp_atk=99eeed27-9831-4c15-8c07-a1c44d38d518&xptdk=99eeed27-9831-4c15-8c07-a1c44d38d518
https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Samsung-Galaxy-S22-Ultra-512GB-256GB-v%C3%A0-128GB-i.1068306860.23981314860?sp_atk=0c5a3220-ffed-4745-b3fc-8adec5d86b68&xptdk=0c5a3220-ffed-4745-b3fc-8adec5d86b68
https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%A1i-Samsung-Galaxy-S23-Ultra-5G-512GB-H%C3%A0ng-Ch%C3%ADnh-H%C3%A3ng-i.1037677547.23977882901?sp_atk=875eb101-e208-4e8d-b7b2-20b5bc95974c&xptdk=875eb101-e208-4e8d-b7b2-20b5bc95974c
https://shopee.vn/Camera-SS-A750-A7-2018-h%C3%A0ng-b%E1%BB%91c-m%C3%A1y-zin-%C4%91%E1%BA%B9p-%C4%91%E1%BB%A7-b%E1%BB%99-i.942951237.23977654410?sp_atk=58ce05e0-77fa-4c70-ba4d-5aed7bb90ec2&xptdk=58ce05e0-77fa-4c70-ba4d-5aed7bb90ec2
https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%A1i-samsung-s21-plus-m%C3%A1y-

## Crawl thông tin sản phẩm

In [7]:
def add_cookies(driver, cookies):
    driver.get("https://shopee.vn")  # Mở Shopee để thêm cookies
    sleep(3)  # Đợi trang tải xong
    for name, value in cookies.items():
        driver.add_cookie({'name': name, 'value': value})
    driver.refresh()  # Tải lại trang sau khi thêm cookies
    print("Cookies added successfully.")

def get_comment(driver, link):
    products = [] # Danh sách để lưu trữ các comment
    driver.get(link)
    sleep(5)  # Đợi trang tải và cookies có hiệu lực

    print(link)

    try:
        # Lấy ngày hiện tại
        current_date = datetime.now().strftime("%Y-%m-%d")
        print(current_date)
        
        product_title = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div.WBVL_7 > span')
        print("Product Title:", product_title)

        brand = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > div.wAMdpk > div > div.page-product__content--left > div.product-detail.page-product__detail > section:nth-child(1) > div > div:nth-child(2) > a')
        print("Brand:", brand)

        current_price = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div:nth-child(3) > div > div > section > div > div.qg2n76')
        
        discounted_price = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div:nth-child(3) > div > div > section > div > div.flex.items-center > div.G27FPf')
        
        discount = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div:nth-child(3) > div > div > section > div > div.flex.items-center > div.o_z7q9')
        print("discount:", discount)
        
        colors = []
        try:
            for i in range(1, 6):  # Lặp từ 1 đến 5
                # Tạo XPath động với giá trị i
                xpath = f'/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[4]/div/div/div/section[1]/div/button[{i}]'
                # Sử dụng XPath để tìm phần tử
                element = driver.find_element(By.XPATH, xpath)
                # Lấy giá trị của thuộc tính aria-label
                aria_label_value = element.get_attribute('aria-label')
                
                # Thêm giá trị vào list colors
                if aria_label_value:  # Kiểm tra xem giá trị có tồn tại không trước khi thêm
                    colors.append(aria_label_value)
        except NoSuchElementException:
            print("Phần tử không được tìm thấy, thoát khỏi vòng lặp.")
        print(colors)


        sold_quanlity = safe_find_text(driver, '/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[2]/div/div[1]', By.XPATH)
        print(sold_quanlity)

        rating_averrage = safe_find_text(driver, '/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[2]/button[1]/div[1]', By.XPATH)
        print(rating_averrage)

        rating_count = safe_find_text(driver, '/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[2]/button[2]/div[1]', By.XPATH)
        print(rating_count)

        store = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.r74CsV.page-product__shop > div.uLQaPg > div > div.fV3TIn')
        print(store)

        product_info = {
            'Platform': 'Shopee',
            'URL': link,
            'Date Crawled': current_date,
            'Store': store,
            'Product Title': product_title,
            'brand': brand,
            'Current Price': current_price,
            'Discounted Price': discounted_price,
            'Discount' : discount,
            'Available Colors': colors,
            'sold_quanlity': sold_quanlity,
            'rating_averrage': rating_averrage,
            'rating_count': rating_count,
        }
        
        products.append(product_info)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        return products

def safe_find_text(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element.text
    except NoSuchElementException:
        return None


In [8]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

# Thêm cookies vào trình duyệt
add_cookies(driver, cookies)

products = []
for link in links:
    products.extend(get_comment(driver, link))

# Sau khi đã lấy xong dữ liệu, đóng trình duyệt
driver.quit()

# Tạo DataFrame từ danh sách comments và xuất ra CSV
comments_df = pd.DataFrame(products)
# Áp dụng hàm đã được import để lọc và thêm cột GB
filtered_comments_df = filter_and_extract_gb_from_df(comments_df)

filtered_comments_df.to_csv('product_smartphone_shopee.csv', mode='a', header=not os.path.exists('product_smartphone.csv'), index=False, encoding='utf-8-sig')
print(filtered_comments_df)

Cookies added successfully.
https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%A1i-Samsung-Galaxy-S23-Ultra-5G-512GB-H%C3%A0ng-Ch%C3%ADnh-H%C3%A3ng-i.1037677547.23977882901?sp_atk=875eb101-e208-4e8d-b7b2-20b5bc95974c&xptdk=875eb101-e208-4e8d-b7b2-20b5bc95974c
2024-03-04
Product Title: Điện Thoại Samsung Galaxy S23 Ultra 5G 512GB - Hàng Chính Hãng
Brand: SAMSUNG
discount: None
Phần tử không được tìm thấy, thoát khỏi vòng lặp.
['Lavender', 'Black', 'Green', 'Cream']
Chưa Có Đánh Giá
None
None
Minh Tuấn Mobile Flagship
https://shopee.vn/(b%E1%BA%A3o-h%C3%A0nh-1-%C4%91%E1%BB%95i-1-trong-v%C3%B2ng-12t)-%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Samsung-Galaxy-S23-Plus-8GB-256GB-8GB-512GB-i.889675785.23951114131?sp_atk=4041f70d-bdad-42c0-8da4-c5e023172f07&xptdk=4041f70d-bdad-42c0-8da4-c5e023172f07
2024-03-04
Product Title: (bảo hành 1 đổi 1 trong vòng 12t) Điện thoại Samsung Galaxy S23 Plus 8GB/256GB - 8GB/512GB
Brand: SAMSUNG
discount: 33% GIẢM
Phần tử không được tìm thấy, thoát khỏi vòng lặp.
['Xanh', 'Đe

In [34]:
df = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Shopee\product_smartphone_shopee.csv")
linkss = df['URL']

In [2]:
linkss = ["https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Apple-iPhone-15-Pro-Max-256GB-i.88201679.20493037179?sp_atk=a2b9c6f5-de4f-448e-9af4-ea692119314b&xptdk=a2b9c6f5-de4f-448e-9af4-ea692119314b"]
linkss

['https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Apple-iPhone-15-Pro-Max-256GB-i.88201679.20493037179?sp_atk=a2b9c6f5-de4f-448e-9af4-ea692119314b&xptdk=a2b9c6f5-de4f-448e-9af4-ea692119314b']

# Get attributes from comments

In [8]:
def add_cookies(driver, cookies):
    driver.get("https://shopee.vn")  # Mở Shopee để thêm cookies
    for name, value in cookies.items():
        driver.add_cookie({'name': name, 'value': value})
    print("Cookies added successfully.")

def get_comment(driver, link):
    comments_list = []  # Danh sách để lưu trữ các comment
    driver.get(link)
    sleep(5)  # Đợi trang tải và cookies có hiệu lực

    print(link)
    # Điều hướng đến Phần Tổng Quan Bình Luận
    try:
        # Đợi cho đến khi phần tổng quan bình luận được tải
        comment_overview = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.product-rating-overview')))
        # Scroll đến phần tổng quan bình luận
        driver.execute_script("arguments[0].scrollIntoView(true);", comment_overview)
        sleep(1)  # Đợi một chút để phần tử scroll vào view

        # Đợi cho đến khi nút bộ lọc sẵn sàng và click vào nó
        filter_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/div[3]/div/div[1]/div[2]/div/div/div[2]/div[2]/div[7]")))
        driver.execute_script("arguments[0].click();", filter_button)  # Sử dụng JS để click tránh ElementClickInterceptedException
        print("Đã click vào bộ lọc.")
        sleep(2)  # Đợi sau khi click vào bộ lọc
    except Exception as e:
        print(f"Không thể tìm thấy hoặc click vào bộ lọc bình luận. Lỗi: {e}")
        return comments_list
    
    while True:        
        try:
            comments = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.shopee-product-rating')))
            for comment_element in comments:
                # Sử dụng find_elements và kiểm tra kích thước danh sách để tránh chờ đợi không cần thiết
                customer_name_elements = comment_element.find_elements(By.CLASS_NAME, 'shopee-product-rating__author-name')
                customer_name = customer_name_elements[0].text.strip() if customer_name_elements else None
                print("customer_name:", customer_name)

                time_comment_elements = comment_element.find_elements(By.CLASS_NAME, 'shopee-product-rating__time')
                time_comment = time_comment_elements[0].text.strip() if time_comment_elements else None
                print("time_comment:", time_comment)

                content_elements = comment_element.find_elements(By.CSS_SELECTOR, "div[style='position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;']")
                content = content_elements[0].text if content_elements else None
                print("content:", content)

                thank_count_elements = comment_element.find_elements(By.CSS_SELECTOR, '.shopee-product-rating__like-count')
                thank_count = thank_count_elements[0].text if thank_count_elements else None
                print("thank_count:", thank_count)

                svg_elements = comment_element.find_elements(By.CSS_SELECTOR, '.shopee-product-rating__rating svg')
                stars = 5 - sum(1 for svg in svg_elements if 'none' in svg.get_attribute('style')) if svg_elements else None
                print("stars:", stars)

                comment_info = {
                    'Platform': 'Shopee',
                    'URL': link,
                    'customer_name': customer_name,
                    'time_comment': time_comment,
                    'content': content,
                    'thank_count': thank_count,
                    'rating': stars
                }
                comments_list.append(comment_info)

            if not end_page(driver):
                print('Finished 1 page.')
                try:
                    next_page = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div > div:nth-child(3) > div.theme--ofs > div > div > div > div.wAMdpk > div > div.page-product__content--left > div:nth-child(2) > div > div > div.product-ratings__list > div.shopee-page-controller.product-ratings__page-controller > button.shopee-icon-button.shopee-icon-button--right')))
                    next_page.click()
                    sleep(5)
                except TimeoutException:
                    print("Reached the end or failed to find the next button.")
                    break
            else:
                print('Last page.')
                break
        except TimeoutException:
            print("Không tìm thấy bình luận cho link này. Chuyển sang link tiếp theo.")
            break
    return comments_list


def end_page(driver):
    try:
        # Tìm container chứa các nút điều hướng trang
        buttons_container = driver.find_element(By.CSS_SELECTOR, 'div.shopee-page-controller.product-ratings__page-controller')
        
        # Lấy tất cả các nút trong container
        buttons = buttons_container.find_elements(By.TAG_NAME, 'button')
        
        # Xác định vị trí của nút "Next" bằng cách tìm nút có selector cụ thể
        next_button = driver.find_element(By.CSS_SELECTOR, "#main > div > div:nth-child(3) > div.theme--ofs > div > div > div > div.wAMdpk > div > div.page-product__content--left > div:nth-child(2) > div > div > div.product-ratings__list > div.shopee-page-controller.product-ratings__page-controller > button.shopee-icon-button.shopee-icon-button--right")
        
        # Xác định vị trí của nút "Next" trong danh sách các nút
        next_button_index = buttons.index(next_button)
        
        # Kiểm tra nếu nút "Prev" tồn tại ngay trước nút "Next"
        if next_button_index > 0 and "shopee-button-solid--primary" in buttons[next_button_index - 1].get_attribute('class'):
            # Nếu có, đây là trang cuối và không cần chuyển trang
            return True
        else:
            # Nếu không, vẫn còn trang để duyệt
            return False
    except NoSuchElementException:
        # Nếu không tìm thấy container hoặc nút, coi như đây là trang cuối
        return True

In [11]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

# Thêm cookies vào trình duyệt
add_cookies(driver, cookies)

comments = []
for link in linkss:
    comments.extend(get_comment(driver, link))

# Sau khi đã lấy xong dữ liệu, đóng trình duyệt
driver.quit()

# Tạo DataFrame từ danh sách comments và xuất ra CSV
comments_df = pd.DataFrame(comments)
comments_df.to_csv('comments_shopee.csv', mode='a', header=not os.path.exists('comments_shopee.csv'), index=False, encoding='utf-8-sig')
print(comments_df)

Cookies added successfully.
https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Apple-iPhone-15-Pro-Max-256GB-i.88201679.20493037179?sp_atk=a2b9c6f5-de4f-448e-9af4-ea692119314b&xptdk=a2b9c6f5-de4f-448e-9af4-ea692119314b
Đã click vào bộ lọc.
customer_name: aringarosa
time_comment: 2023-12-15 14:47 | Phân loại hàng: Titan - Đen
content: Canh sale 0h 12.12 may mắn săn được mã giảm 3 triệu, shopee cũng giao hàng rất nhanh, ngay hôm sau đã nhận được rồi. Cảm ơn shop nhiều nha.
thank_count: 139
stars: 5
customer_name: italiadang99
time_comment: 2023-11-21 14:35 | Phân loại hàng: Titan - Tự Nhiên
content: Hàng chuẩn VNA. Lần đầu ship giá trị cao đợi 3 ngày mà dài như 3 tháng lun. Máy kh lỗi, sang xịn mịn. Hoá đơn điện tử gửi trước 1 ngày nhận hàng
thank_count: 332
stars: 5
customer_name: vantruong33
time_comment: 2023-10-10 09:18 | Phân loại hàng: Titan - Xanh
content: Lần đầu mua đồ giá trị cao trên shoppe khá là run, nhưng ship hàng nhanh e mới mua trưa bửa trước là chiều hôm sau nhận được rồ